In [2]:
import torch

In [60]:
seq_k = torch.tensor([[1, 2, 0, 0, 0, 0], 
                      [4, 5, 6, 0, 0, 0], 
                      [7, 8, 9, 10, 0, 0], 
                      [10, 11, 12, 13, 14, 0], 
                      [13, 14, 15, 0, 0, 0]])

seq_q = torch.tensor([[2, 3, 4, 0, 0],
                        [5, 6, 0, 0, 0],
                        [8, 9, 10, 11, 0],
                        [11, 0, 0, 0, 0],
                        [14, 15, 16, 17, 10]])

In [61]:
batch_size, len_q = seq_q.size() # batch_size x len_q
batch_size, len_k = seq_k.size() # batch_size x len_k

In [62]:
print(seq_q.shape)
print(seq_k.shape)

torch.Size([5, 5])
torch.Size([5, 6])


In [63]:
seq_k.data.eq(0)

tensor([[False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False,  True,  True,  True]])

In [64]:
seq_k.data.eq(0).shape

torch.Size([5, 6])

In [65]:
pad_mask = seq_k.data.eq(0).unsqueeze(1)

In [66]:
pad_mask.shape

torch.Size([5, 1, 6])

In [67]:
padding_mask = pad_mask.expand(batch_size, len_q, len_k)

In [68]:
padding_mask.shape

torch.Size([5, 5, 6])

In [71]:
padding_mask[1, :, :]

tensor([[False, False, False,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False,  True,  True,  True]])

In [73]:
padding_mask[2, :, :]

tensor([[False, False, False, False,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False,  True,  True]])

In [76]:
padding_mask[:, 0, :]

tensor([[False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False,  True,  True,  True]])

In [79]:
padding_mask[:, 1, :]

tensor([[False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False,  True,  True,  True]])

In [80]:
look_ahead_mask = torch.ones(len_q, len_q).triu(1) # seq_len x seq_len


In [81]:
look_ahead_mask

tensor([[0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.]])

In [82]:
look_ahead_mask.shape

torch.Size([5, 5])

In [83]:
look_ahead_masking = look_ahead_mask.unsqueeze(0).expand(batch_size, len_q, len_q)

In [85]:
look_ahead_masking.shape

torch.Size([5, 5, 5])

In [91]:
look_ahead_masking[2, :, :]

tensor([[0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.]])

In [93]:
look_ahead_masking[2, 2, :]

tensor([0., 0., 0., 1., 1.])

In [40]:
import torch.nn as nn
import math
import torch.nn.functional as F

In [48]:
d_model = 512
a_2 = nn.Parameter(torch.ones(d_model))
b_2 = nn.Parameter(torch.zeros(d_model))

In [50]:
a_2.shape

torch.Size([512])

In [52]:
b_2.shape

torch.Size([512])

In [55]:
start_symbol = 1
src = torch.tensor([[1, 2, 3, 4, 5, 0],
                    [4, 5, 6, 7, 8, 0],
                    [7, 8, 9, 10, 11, 0],
                    [10, 11, 12, 13, 14, 0],
                    [13, 14, 15, 16, 17, 0]])

In [58]:
dec_input = torch.zeros(src.size(0), 1).fill_(start_symbol).type_as(src.data) # batch_size x 1
dec_output = dec_input

In [59]:
dec_output

tensor([[1],
        [1],
        [1],
        [1],
        [1]])

In [62]:
def create_look_ahead_mask(seq: torch.Tensor) -> torch.Tensor:
    batch_size, seq_len = seq.size() # batch_size x seq_len
    look_ahead_mask = torch.ones(seq_len, seq_len).triu(1) # seq_len x seq_len

    # 1 mean masked
    return look_ahead_mask.unsqueeze(0).expand(batch_size, seq_len, seq_len) # batch_size x seq_len x seq_len



In [63]:
look_ahead_mask = create_look_ahead_mask(dec_output)

In [18]:
look_ahead_mask

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])

In [65]:
look_ahead_mask.shape

torch.Size([5, 1, 1])

In [11]:
B, T, C, = 4, 8, 2
x = torch.randn(B,T,C) #shape (B,T,C)

In [14]:
x.shape

torch.Size([4, 8, 2])

In [15]:
x

tensor([[[-1.7581,  0.8704],
         [ 1.9663, -0.2711],
         [-0.3641,  1.0965],
         [ 1.2530,  0.5127],
         [ 0.2694,  0.5246],
         [-0.5033, -1.1301],
         [ 0.5210,  0.0061],
         [ 1.3937, -0.0289]],

        [[-0.8381, -0.3315],
         [ 0.5582,  0.6497],
         [ 0.4310,  0.6970],
         [-0.7077,  0.1202],
         [ 0.7559, -1.3130],
         [ 0.2301, -0.4101],
         [ 0.1752, -0.4074],
         [ 1.5727, -0.9901]],

        [[ 0.1028,  0.2099],
         [ 1.6814, -0.0090],
         [ 0.5263,  1.4878],
         [ 0.9548,  0.4714],
         [ 0.4682,  1.2507],
         [ 0.6670,  0.5630],
         [-1.7343,  1.5960],
         [ 1.1868,  0.3576]],

        [[ 0.3733, -0.2750],
         [ 1.0779,  1.3829],
         [ 1.0023, -0.5159],
         [ 0.5725,  1.4167],
         [ 0.3471, -0.2192],
         [-0.4475,  0.3757],
         [ 2.0864, -0.1763],
         [ 0.6250,  0.1615]]])

In [16]:
#attention matrix (lower triangular), a mask used to only show previous items to predict next item
wei = torch.tril(torch.ones((T,T)))

In [17]:
wei

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [21]:
wei.shape

torch.Size([8, 8])

In [20]:
wei.sum(dim=1, keepdim=True).shape

torch.Size([8, 1])

In [22]:
w2 = wei/wei.sum(dim=1, keepdim=True)

In [36]:
w2

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [28]:
out1 = torch.matmul(w2, x)
out2 = w2 @ x

In [32]:
out1.shape

torch.Size([4, 8, 2])

In [33]:
out2.shape

torch.Size([4, 8, 2])

In [34]:
out1 == out2

tensor([[[True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True],
         [True, True]]])

In [38]:
x.shape

torch.Size([4, 8, 2])

In [41]:
class CausalSelfAttention(nn.Module):
    def __init__(
        self,
        d,
        H,
        T,
        bias=False,
        dropout=0.2,
    ):
        """
        Arguments:
        d: size of embedding dimension
        H: number of attention heads
        T: maximum length of input sequences (in tokens)
        bias: whether or not to use bias in linear layers
        dropout: probability of dropout
        """
        super().__init__()
        assert d % H == 0

        # key, query, value projections for all heads, but in a batch
        # output is 3X the dimension because it includes key, query and value
        self.c_attn = nn.Linear(d, 3*d, bias=bias)

        # projection of concatenated attention head outputs
        self.c_proj = nn.Linear(d, d, bias=bias)

        # dropout modules
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)
        self.H = H
        self.d = d

        # causal mask to ensure that attention is only applied to
        # the left in the input sequence
        self.register_buffer("mask", torch.tril(torch.ones(T, T))
                                    .view(1, 1, T, T))

    def forward(self, x):
        B, T, _ = x.size() # batch size, sequence length, embedding dimensionality

        # compute query, key, and value vectors for all heads in batch
        # split the output into separate query, key, and value tensors
        q, k, v  = self.c_attn(x).split(self.d, dim=2) # [B, T, d]

        # reshape tensor into sequences of smaller token vectors for each head
        k = k.view(B, T, self.H, self.d // self.H).transpose(1, 2) # [B, H, T, d // H]
        q = q.view(B, T, self.H, self.d // self.H).transpose(1, 2)
        v = v.view(B, T, self.H, self.d // self.H).transpose(1, 2)

        # compute the attention matrix, perform masking, and apply dropout
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1))) # [B, H, T, T]
        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_dropout(att)

        # compute output vectors for each token
        y = att @ v # [B, H, T, d // H]

        # concatenate outputs from each attention head and linearly project
        y = y.transpose(1, 2).contiguous().view(B, T, self.d)
        y = self.resid_dropout(self.c_proj(y))
        return y

In [44]:
d = 512
H = 8
T = 1024
bias=True
dropout=0.2

In [45]:
a = CausalSelfAttention(d, H, T, bias=bias, dropout=dropout)

In [48]:
a.mask.shape

torch.Size([1, 1, 1024, 1024])

In [50]:
a.mask[:,:,:T,:T].shape

torch.Size([1, 1, 1024, 1024])

In [2]:
import torch

In [12]:
b = 32
t = 512
n_tok = 8
dim = 1024

probs = torch.randn(b*t, n_tok) # shape (b*t, n_tok)

In [20]:
tok_idx = 4
expert_idx = 2

p = probs[tok_idx, expert_idx, None]

In [19]:
p.shape

torch.Size([])

In [13]:
x = torch.randn(b*t, dim) # shape (b*t, dim)

In [15]:
x[tok_idx].shape

torch.Size([1024])

In [17]:
(x[tok_idx]*p).shape

torch.Size([1024])